In [13]:
import librosa
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os,glob
import cv2
import numpy as np
from os import listdir,makedirs
from os.path import isfile,join
import sys
from sklearn.model_selection import train_test_split


In [14]:
data = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
final_list = []

In [17]:
#Checked If processed_images folder is exist or not
# if not os.path.exists('./processed_images'):
#             os.mkdir('./processed_images')
for folder_index in range(10):
    path = 'audio_spectograms/'+str(folder_index)
#     if not os.path.exists('./processed_images/'+str(folder_index)):
#             os.mkdir('./processed_images/'+str(folder_index))
#     else:
#             print('Images are already preprocessed!')
#             sys.exit(0)  
    for image_file in os.listdir(path):
        #if processed_images directory exists, then save the image in that directory
        image = cv2.imread(os.path.join(path,image_file))
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray,(256,256))
        #convert image to float32
        resize = np.asarray(resize).astype(np.float32)
        normalize = resize/255.0
        #cv2.imwrite('.\processed_images\\'+str(folder_index)+'\\'+image_file,normalize)
        #append normalize photos and their labels to the list
        final_list.append([normalize,int(image_file.split('-')[1])]) 
        #print normalize part from list
       
    # print(final_list)
    
          
                
    

In [19]:
model = tf.keras.Sequential()
y = [i[1] for i in final_list] #labels-class
x = [i[0] for i in final_list] #features-image
x = np.asarray(x).astype(np.float32)
y = np.array(y)
x_train,x_temp,y_train,y_temp = train_test_split(x,y,train_size = 0.8)
x_test,x_val,y_test,y_val = train_test_split(x_temp,y_temp,test_size=0.5)
# print(len(x_train))
# print(len(x_test))
# print(len(x_val))
model.add(tf.keras.layers.Conv2D(32,
                                 kernel_size=(3,3),strides=(1,2),
                                 padding="same",activation="relu",
                                 input_shape=(256,256,1)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64,
                                 kernel_size=(3,3),strides=(1,2),
                                 padding="same",activation="relu"
                                 ))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64,activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64,activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10,activation="softmax"))

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss="sparse_categorical_crossentropy",
              metrics=["Accuracy"])
results = model.fit(x_train,y_train,
                    batch_size=64,
                    epochs=20,verbose=1,
                    validation_data=(x_val,y_val))